# OpenCV

## Imports

In [69]:
import cv2
import argparse
import numpy as np
import imutils
cv2.__version__

'3.4.0'

## Play Video from a File

In [3]:
cap = cv2.VideoCapture('RedCar.avi')


if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():

    ret, frame = cap.read()
    
    if ret:
        
        cv2.imshow('Frame', frame)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Find Brightest Spot with Gaussian Blur

In [4]:
cap = cv2.VideoCapture('RedCar.avi')


if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():

    ret, frame = cap.read()
    
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (11,11), 0)
        
        (minVal, maxVal, minLoc, maxLoc) = cv2.minMaxLoc(gray)
        #cv2.circle(frame, minLoc, 5, (255, 0, 0), 2)
        cv2.circle(frame, maxLoc, 41, (255, 0, 0), 2)

        
        cv2.imshow('Frame', frame)
        cv2.imshow('Gray', gray)
        
        
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Find Colors

In [5]:
cap = cv2.VideoCapture('RedCar.avi')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():

    ret, frame = cap.read()
    
    if ret:
        """
        for (lower, upper) in boundaries:
            lower = np.array(lower, dtype='uint8')
            upper = np.array(upper, dtype='uint8')
            
            mask = cv2.inRange(frame, lower, upper)
            output = cv2.bitwise_and(frame, frame, mask = mask)
        """
        
        #set lower and upper limits in bgr order
        lower = np.array([154,147,203], dtype='uint8')
        upper = np.array([204,217,255], dtype='uint8')
        
        mask = cv2.inRange(frame, lower, upper)
        output = cv2.bitwise_and(frame, frame, mask = mask)
        
        #cv2.imshow('Frame', frame)
        cv2.imshow("images", np.hstack([frame, output]))
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Find Colors with IPython Interact

In [6]:
cap = cv2.VideoCapture('RedCar.avi')

"""
#Define the color ranges in BGR order
boundaries = [
    ([154, 147, 203], [255,255,255]), #Red
    ([0, 0, 0], [255, 255, 255]), #Blue
    ([0, 0, 0], [255, 255, 255]), #Yellow
    ([0, 0, 0], [255, 255, 255]) #Gray
]

def set_threshold(b, g, r):
    return np.array([b,g,r], dtype='uint8')

lower = interact(set_threshold, b=154, g=147, r=203)
"""
if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():

    ret, frame = cap.read()
    
    if ret:
        """
        for (lower, upper) in boundaries:
            lower = np.array(lower, dtype='uint8')
            upper = np.array(upper, dtype='uint8')
            
            mask = cv2.inRange(frame, lower, upper)
            output = cv2.bitwise_and(frame, frame, mask = mask)
        """
        #print(type(lower))
        #print(type(interact(set_threshold, b=154, g=147, r=203)))
        #lower = set_threshold, b=154, g=147, r=203
        #lower = set_threshold(154,147,203)
        #upper = set_threshold(204,217,255)
        #set lower and upper limits in bgr order
        
        lower = np.array([154,147,203], dtype='uint8')  
        upper = np.array([199,212,250], dtype='uint8')
        
        mask = cv2.inRange(frame, lower, upper)
        output = cv2.bitwise_and(frame, frame, mask = mask)
        
        #cv2.imshow('Frame', frame)
        cv2.imshow("images", np.hstack([frame, output]))
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Edge Detection

### Auto-Canny

In [9]:
def auto_canny(frame, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(frame)
 
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(frame, lower, upper)
 
    # return the edged image
    return edged

In [15]:
cap = cv2.VideoCapture('RedCar.avi')


if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():

    ret, frame = cap.read()
    
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (3, 3), 0)
        
        wide = cv2.Canny(blurred, 10, 200)
        tight = cv2.Canny(blurred, 280, 300)
        auto = auto_canny(blurred)
 
        cv2.imshow('Original', frame)
        cv2.imshow('wide', wide)
        cv2.imshow('tight', tight)
        cv2.imshow('auto', auto)
        #cv2.imshow("Edges", np.hstack([wide, tight, auto]))
    
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Shape Detection

In [49]:
class ShapeDetector:
    def __init__(self):
        pass
 
    def detect(self, c):
        # initialize the shape name and approximate the contour
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        # if the shape is a triangle, it will have 3 vertices
        if len(approx) == 3:
            shape = "triangle"
 
        # if the shape has 4 vertices, it is either a square or
        # a rectangle
        elif len(approx) == 4:
            # compute the bounding box of the contour and use the
            # bounding box to compute the aspect ratio
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)
 
            # a square will have an aspect ratio that is approximately
            # equal to one, otherwise, the shape is a rectangle
            shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"
 
        # if the shape is a pentagon, it will have 5 vertices
        elif len(approx) == 5:
            shape = "pentagon"
 
        # otherwise, we assume the shape is a circle
        else:
            shape = "circle"
 
        # return the name of the shape
        return shape

In [86]:
cap = cv2.VideoCapture('RedCar.avi')


if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():

    ret, frame = cap.read()
    
    if ret:
        
        resized = imutils.resize(frame, width=3000)
        ratio = frame.shape[0] / float(resized.shape[0])
        
        gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        thresh = cv2.threshold(blurred, 210, 255, cv2.THRESH_BINARY)[1]
        
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        cnts = cnts[0] if imutils.is_cv2() else cnts[1]
        sd = ShapeDetector()
        
        try:
            for c in cnts:
                # compute the center of the contour, then detect the name of the
                # shape using only the contour
                M = cv2.moments(c)
                cX = int((M["m10"] / M["m00"]) * ratio)
                cY = int((M["m01"] / M["m00"]) * ratio)
                shape = sd.detect(c)

                # multiply the contour (x, y)-coordinates by the resize ratio,
                # then draw the contours and the name of the shape on the image
                c = c.astype("float")
                c *= ratio
                c = c.astype("int")
                if shape == 'rectangle':   
                    cv2.drawContours(frame, [c], -1, (0, 255, 0), 2)
                    cv2.putText(frame, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        except:
            pass
        
        cv2.imshow('Frame', frame)
        cv2.imshow('thresh', thresh)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## All Together

In [48]:
cap = cv2.VideoCapture('RedCar.avi')


if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():

    ret, frame = cap.read()
    
    if ret:
        
        lower = np.array([154,147,203], dtype='uint8')
        upper = np.array([174,167,255], dtype='uint8')
        
        mask = cv2.inRange(frame, lower, upper)
        output = cv2.bitwise_and(frame, frame, mask = mask)
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (3, 3), 0)
        
        tight_canny = cv2.Canny(output, 280, 300)
        
        image, contours, hierarchy = cv2.findContours(tight_canny,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        
        contour_list = []
        for contour in contours:
            approx = cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True)
            area = cv2.contourArea(contour)
            if ((len(approx) > 1) & (area > 1) ):
                contour_list.append(contour)
        
        output = cv2.drawContours(image, contour_list, -1, (255,255,255), 2)
        
        cv2.imshow('Frame', output)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Capture Video from a Camera

In [7]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    
    ret, frame = cap.read()
    
    cv2.imshow('Frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()